# Crop Screenshots Notebook

This notebook is designed to crop JPEG images generated from PNG screenshots. 
Using the Python Imaging Library (Pillow), it processes each JPEG image located in a specified input directory, 
applies a predefined bounding box to crop the images, and saves the cropped versions in a designated output directory. 
The cropping is particularly useful for isolating specific parts of the images, such as selections or highlights, 
to focus on relevant content.

## Features:
- Automatically searches for JPEG images in the input directory.
- Crops images based on specified pixel coordinates for a bounding box.
- Saves the cropped images with a modified filename to differentiate them from the originals.
- Produces console output for each cropped image, indicating conversion success.

Ensure that the input directory contains JPEG images before running the notebook, and modify the 
bounding box dimensions as needed to achieve the desired cropping effect.

## Setup:

In [1]:
# Import necessary libraries
import os
from PIL import Image
import matplotlib.pyplot as plt

In [2]:
# Define any bounding boxes to be used

# Bounding box (in pixels)
SQUAD_BATTLES_SELECTION_BBOX = (140, 363, 140 + 290, 363 + 545)  # (left, upper, right, lower)

In [3]:
# Set up input and output directories for image processing
input_dir = '../screenshots/jpg'
output_dir = '../screenshots/jpg/cropped'
os.makedirs(output_dir, exist_ok=True)

### Functions:
Define different cropping functions below.

In [4]:
# Function to crop to centered 1080p
def crop_center_to_1080p(image_path):
    img = Image.open(image_path)
    target_width = 1920
    target_height = 1080
    width, height = img.size

    # Calculate the cropping box
    left = (width - target_width) / 2
    top = (height - target_height) / 2
    right = (width + target_width) / 2
    bottom = (height + target_height) / 2

    cropped_img = img.crop((left, top, right, bottom))
    return cropped_img

# Function to crop based on a predefined bounding box
def crop_with_bbox(image_path, bbox):
    img = Image.open(image_path)
    cropped_img = img.crop(bbox)
    return cropped_img

# Function to save cropped images
def save_image(image, file_path):
    image.save(file_path, 'JPEG', quality=90)
    print(f"Saved: {file_path}")

# Function for batch cropping with bounding box
def batch_crop_with_bbox(input_dir, output_dir, bbox):
    os.makedirs(output_dir, exist_ok=True)
    for file_name in os.listdir(input_dir):
        if file_name.lower().endswith('.jpg'):
            jpg_file_path = os.path.join(input_dir, file_name)
            base_name = os.path.splitext(file_name)[0]
            cropped_file_path = os.path.join(output_dir, f"{base_name}_cropped.jpg")

            cropped_img = crop_with_bbox(jpg_file_path, bbox)
            save_image(cropped_img, cropped_file_path)

# Function for batch cropping to centered 1080p
def batch_crop_center_to_1080p(input_dir, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    for file_name in os.listdir(input_dir):
        if file_name.lower().endswith('.jpg'):
            jpg_file_path = os.path.join(input_dir, file_name)
            base_name = os.path.splitext(file_name)[0]
            cropped_file_path = os.path.join(output_dir, f"{base_name}_cropped.jpg")

            cropped_img = crop_center_to_1080p(jpg_file_path)
            save_image(cropped_img, cropped_file_path)

## Use Cases:

In [ ]:
# 1. Batch crop to centered 1080p
batch_crop_center_to_1080p(input_dir, output_dir)

In [6]:
# 2. Batch crop using bounding box
batch_crop_with_bbox(input_dir, output_dir, SQUAD_BATTLES_SELECTION_BBOX)